In [1]:
!pip3 install cmake
!pip3 install dlib

In [2]:
!pip3 install imutils

  Preparing metadata (setup.py) ... done
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25836 sha256=2a827cf2b257c040587db4446f4b585e04bc30bedab205d598ad6d71246b09f6
  Stored in directory: /Users/kunjshah/Library/Caches/pip/wheels/85/cf/3a/e265e975a1e7c7e54eb3692d6aa4e2e7d6a3945d29da46f2d7
Successfully built imutils


In [2]:
from imutils import face_utils
import imutils
from imutils.video import VideoStream
from imutils import face_utils
from imutils.video import FPS
import dlib
import cv2
import time
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
from pygame import mixer

pygame 2.3.0 (SDL 2.24.2, Python 3.10.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
face_detector = dlib.get_frontal_face_detector()

face_landmark_predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

In [4]:
model_folder = "models/"
model = load_model(f"{model_folder}inception3_model0414.h5")
mixer.init()

In [5]:
cap = cv2.VideoCapture(1)

In [6]:
def eye_preprocessing(eye):
    eye = cv2.resize(eye,(80,80))
    eye = eye/255
    eye = eye.reshape(80,80,3)
    eye = np.expand_dims(eye, axis=0)
    return eye

In [ ]:
try:
    cap = cv2.VideoCapture(1)

    while True:
        # Capture a frame from the camera
        ret, frame = cap.read()

        # Convert the frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the frame
        faces = face_detector(gray)

        # Loop over the detected faces
        for face in faces:
            # Get the facial landmarks
            landmarks = face_landmark_predictor(gray, face)

            # Extract the eye regions using the landmarks
            left_eye = [(landmarks.part(n).x, landmarks.part(n).y) for n in range(36, 42)]
            right_eye = [(landmarks.part(n).x, landmarks.part(n).y) for n in range(42, 48)]

            # Extract the eye region bounding boxes with a little extra margin
            left_eye_rect = (min(left_eye, key=lambda x: x[0])[0] - 30, min(left_eye, key=lambda x: x[1])[1] - 30,
                             max(left_eye, key=lambda x: x[0])[0] + 30, max(left_eye, key=lambda x: x[1])[1] + 30)
            right_eye_rect = (min(right_eye, key=lambda x: x[0])[0] - 30, min(right_eye, key=lambda x: x[1])[1] - 30,
                              max(right_eye, key=lambda x: x[0])[0] + 30, max(right_eye, key=lambda x: x[1])[1] + 30)

            # Draw squares around the eye regions
            cv2.rectangle(frame, (left_eye_rect[0], left_eye_rect[1]),
                          (left_eye_rect[2], left_eye_rect[3]),
                          (0, 255, 0), 2)
            cv2.rectangle(frame, (right_eye_rect[0], right_eye_rect[1]),
                          (right_eye_rect[2], right_eye_rect[3]),
                          (0, 255, 0), 2)

            left_eye_frame = frame[left_eye_rect[0]:left_eye_rect[1],left_eye_rect[2]:left_eye_rect[3]]
            right_eye_frame = frame[right_eye_rect[0]:right_eye_rect[1],right_eye_rect[2]:right_eye_rect[3]]

            # print("left_eye", left_eye)
            # print("right", right_eye)

#             left_eye_frame = eye_preprocessing(left_eye_frame)
#             right_eye_frame = eye_preprocessing(right_eye_frame)

#             left_eye_prediction = model.predict(left_eye_frame)
#             right_eye_prediction = model.predict(right_eye_frame)
#                 if left_eye_prediction[0][0]>0.70:
#                     cv2.putText(frame,'closed',(200,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,255,255),
#                            thickness=1,lineType=cv2.LINE_AA)
#                 else:
#                     cv2.putText(frame,'open',(200,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,255,255),
#                            thickness=1,lineType=cv2.LINE_AA)

                    # Display the frame with eye detections
            cv2.imshow("Eye Detection", frame)

            # Exit the loop on 'q' key press
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
except Exception as e:
    print(e)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

cv2.destroyAllWindows()
cv2.waitKey(1)

2023-04-24 15:48:44.404 python[58083:5651899] IMKClient Stall detected, *please Report* your user scenario attaching a spindump (or sysdiagnose) that captures the problem - (imkxpc_bundleIdentifierWithReply:) block performed very slowly (3.11 secs).


In [5]:
try:
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
    mixer.init()
    sound= mixer.Sound("../alarm.wav")
    cap = cv2.VideoCapture(0)
    Score = 0
    while True:
        ret, frame = cap.read()
        height,width = frame.shape[0:2]
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces= face_cascade.detectMultiScale(gray, scaleFactor= 1.2, minNeighbors=3)
        eyes= eye_cascade.detectMultiScale(gray, scaleFactor= 1.1, minNeighbors=1)

        cv2.rectangle(frame, (0,height),(400,height),(0,0,0),thickness=cv2.FILLED)

        for (x,y,w,h) in faces:
            cv2.rectangle(frame,pt1=(x,y),pt2=(x+w,y+h), color=(255,0,0), thickness=3)

        for (ex,ey,ew,eh) in eyes:
            print("eyes",ex,ey,ew,eh)
            cv2.rectangle(frame,pt1=(ex,ey),pt2=(ex+ew,ey+eh), color=(255,0,0), thickness=3)

            # preprocessing steps
            eye= frame[ey:ey+eh,ex:ex+ew]
            eye= cv2.resize(eye,(80,80))
            eye= eye/255
            eye= eye.reshape(80,80,3)
            eye= np.expand_dims(eye,axis=0)
            # preprocessing is done now model prediction
#             prediction = model.predict(eye)

#             # if eyes are closed
#             if prediction[0][0]>0.70:
#                 cv2.putText(frame,'closed',(200,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,255,255),
#                            thickness=1,lineType=cv2.LINE_AA)
#                 cv2.putText(frame,'Score'+str(Score),(300,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,255,255),
#                            thickness=1,lineType=cv2.LINE_AA)
#                 Score=Score+1
#                 if(Score>15):
#                     try:
#                         sound.play()
#                     except:
#                         pass

#             # if eyes are open
#             elif prediction[0][1]>0.90:
#                 cv2.putText(frame,'open',(10,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,255,255),
#                            thickness=1,lineType=cv2.LINE_AA)      
#                 cv2.putText(frame,'Score'+str(Score),(100,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,255,255),
#                            thickness=1,lineType=cv2.LINE_AA)
#                 Score = Score - 3
#                 if Score < 0:
#                     Score = 0


        cv2.imshow('frame',frame)
        if cv2.waitKey(33) & 0xFF==ord('q'):
            break
except Exception as e:
    print(e)
    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)
        
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

eyes 905 442 29 29
eyes 882 316 69 69
eyes 891 322 68 68
eyes 154 260 34 34
eyes 878 306 92 92
eyes 912 448 21 21
eyes 893 316 66 66
eyes 874 310 68 68
eyes 224 685 22 22
eyes 859 306 70 70
eyes 829 284 122 122
eyes 762 302 102 102
eyes 225 682 26 26
eyes 758 316 54 54
eyes 850 300 84 84
eyes 223 682 26 26
eyes 157 258 32 32
eyes 855 300 77 77
eyes 767 312 61 61
eyes 757 304 106 106
eyes 656 312 101 101
eyes 778 314 61 61
eyes 853 302 81 81
eyes 646 312 106 106
eyes 680 334 42 42
eyes 225 682 23 23
eyes 776 314 63 63
eyes 854 300 81 81
eyes 683 334 43 43
eyes 640 307 115 115
eyes 857 304 75 75
eyes 774 314 64 64
eyes 647 307 111 111
eyes 164 265 21 21
eyes 772 310 73 73
eyes 857 305 77 77
eyes 681 330 47 47
eyes 643 306 118 118
eyes 932 625 22 22
eyes 162 265 22 22
eyes 685 335 39 39
eyes 674 340 37 37
eyes 775 313 64 64
eyes 859 303 77 77
eyes 161 262 24 24
eyes 779 314 63 63
eyes 679 344 32 32
eyes 861 305 73 73
eyes 662 313 84 84
eyes 158 259 29 29
eyes 777 309 69 69
eyes 854 304 86

-1